Notebook for comparing results with open rocket and calculating data for far-out report

In [ ]:
from rocketpy import Rocket, Environment, Flight, LiquidMotor, Fluid, CylindricalTank, MassFlowRateBasedTank, MassBasedTank
from math import exp
from datetime import datetime, timedelta
from CoolProp.CoolProp import PropsSI
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import excel_sheet_functions as ex

In [ ]:
#open rocket from Oskar
env = Environment( 
    latitude=28.61,
    longitude=-80.6, # from OR
)
env.set_atmospheric_model(
    type="custom_atmosphere",
    pressure=None,
    temperature=300,
    wind_u=[
        (0, -4.47), #wind set to be similar ot open rocket
        (1000, -4.47), 
        (5000, -4.47),
    ],
    wind_v=[
        (0, 0), 
        (1000, 0), 
        (5000, 0), 
    ],
)
env.set_elevation(0) # from OR
env.max_expected_height = 10000
# env.plots.atmospheric_model()

# env.info()

In [ ]:
# Stałe dla turbulencji
external_tank_diameter = 0.2
tank_height = 1.13
thickness_tank = 0.005
thickness_piston = 0.01

# Parametry, które potrzebuję
p_0 = 63e5 #ciśnienie początkowe
piston_position = 0.85
total_oxidizer_mass = 18
flux_time = 11.2 # the start time will be useful as well
# csv z masą utleniacza i paliwa w czasie
ethanol_temperature = 300
# Parametry, które zakładem
gas_initial_mass_fuel = 0

# Parametry, które wyliczam
N20_liq_density = PropsSI("D", "P", p_0, "Q", 0, "NitrousOxide")
N20_gas_density = PropsSI("D", "P", p_0, "Q", 1, "NitrousOxide")
ethanol_liq_density = PropsSI("D", "P", p_0-1e5, "T", ethanol_temperature, "Ethanol") # PropSI pokazuje 63e5 powyżej krytycznego, dlatego odejmuję
ethanol_gas_density = PropsSI("D", "P", p_0-1e5, "Q", 1, "Ethanol")

volume_tank = 0.25*np.pi*(external_tank_diameter-2*thickness_tank)**2*tank_height
volume_piston = 0.25*np.pi*(external_tank_diameter-2*thickness_tank)**2*thickness_piston
volume_oxidizer = piston_position*volume_tank
volume_fuel = volume_tank - volume_oxidizer - volume_piston

gas_initial_mass_ox = (volume_oxidizer - (total_oxidizer_mass / N20_liq_density)) / (1/N20_gas_density - 1/N20_liq_density)
liquid_initial_mass_ox = total_oxidizer_mass - gas_initial_mass_ox
liquid_initial_mass_fuel = volume_fuel * ethanol_liq_density

# Parametry stricte pod tank_geometry
tank_radius = (external_tank_diameter - 2*thickness_tank) / 2
adjusted_height_ox = piston_position*tank_height
adjusted_height_fuel = tank_height - adjusted_height_ox - thickness_piston

# Dla sprawdzenia czy wychodzi mi to samo co volume_oxidizer i volume_fuel
fuel_tank_volume = np.pi * tank_radius**2 * adjusted_height_fuel
oxidizer_tank_volume = np.pi * tank_radius**2 * adjusted_height_ox

In [ ]:
oxidizer_liq = Fluid(name="N2O_l", density=N20_liq_density)
oxidizer_gas = Fluid(name="N2O_g", density=N20_gas_density)
fuel_liq = Fluid(name="ethanol_l", density=ethanol_liq_density) 
fuel_gas = Fluid(name="ethanol_g", density=ethanol_gas_density)

In [ ]:
# Tank geometry
oxidizer_tank_geometry = CylindricalTank(
    radius=tank_radius,
    height=adjusted_height_ox,
)
fuel_tank_geometry = CylindricalTank(
    radius=tank_radius,
    height=adjusted_height_fuel,
)

In [ ]:
# Define tanks
mass_flow_rate_liq = round(liquid_initial_mass_ox/flux_time, 2) - 0.005 # waiting for csv
mass_flow_rate_gas = round(gas_initial_mass_ox/flux_time - 0.005, 2) # waiting for csv

oxidizer_tank = MassFlowRateBasedTank(
    name="oxidizer tank",
    geometry=oxidizer_tank_geometry,
    flux_time=(1, flux_time+1), 
    initial_liquid_mass=liquid_initial_mass_ox, 
    initial_gas_mass=gas_initial_mass_ox,
    liquid_mass_flow_rate_in=0,
    liquid_mass_flow_rate_out=mass_flow_rate_liq, 
    gas_mass_flow_rate_in=0,
    gas_mass_flow_rate_out=mass_flow_rate_gas, 
    liquid=oxidizer_liq,
    gas=oxidizer_gas,
)

In [ ]:
#Fuel tank
fuel_mass_flow_rate = liquid_initial_mass_fuel / flux_time - 0.01
fuel_tank = MassFlowRateBasedTank(
    name="fuel tank",
    geometry=fuel_tank_geometry,
    flux_time=(1, flux_time+1),
    initial_liquid_mass=liquid_initial_mass_fuel-0.00001, #Same as above, only guess
    initial_gas_mass=gas_initial_mass_fuel,
    liquid_mass_flow_rate_in=0,
    liquid_mass_flow_rate_out=fuel_mass_flow_rate, #heuristics
    gas_mass_flow_rate_in=0,
    gas_mass_flow_rate_out=0,
    liquid=fuel_liq,
    gas=fuel_gas,
)

In [ ]:
z4000 = LiquidMotor(
    thrust_source=".\\data\\AGH-SS_Z4000-10sBurn-optimal.eng", #From tests
    dry_mass=2.7,
    dry_inertia=(0.02143, 0.02143, 0.005535), #This should be calculated using CAD, here I use estimations
    nozzle_radius=0.036, #From technical report
    center_of_dry_mass_position=0.144, #Estimated from openrocket
    nozzle_position=0,
    burn_time=14.4,
    coordinate_system_orientation="nozzle_to_combustion_chamber",
)
z4000.add_tank(tank=oxidizer_tank, position=1.285) #From nozzle to center of the tank
z4000.add_tank(tank=fuel_tank, position=2.01)

In [ ]:
trb = Rocket(
    radius=0.1,
    mass=58.367, 
    inertia=(75.502, 75.502, 0.43), 
    power_off_drag=".\\data\\powerondrag.csv",  
    power_on_drag='.\\data\\powerondrag.csv',
    center_of_mass_without_motor=2.75, 
    coordinate_system_orientation="nose_to_tail", 
)
trb.add_motor(z4000, position=4.49)

In [ ]:
#Aerodynamic surfaces taken from openrocket
nose_cone = trb.add_nose(
    length=0.7, kind="lvhaack", position=0
)

fin_set = trb.add_trapezoidal_fins(
    n=4,
    root_chord=0.287,
    tip_chord=0.059,
    span=0.202,
    sweep_length=0.228,
    position=4.21,
    cant_angle=0,
)

tail = trb.add_tail(
    top_radius=0.1, bottom_radius=0.065, length=0.287, position=4.21
)

rail_buttons = trb.set_rail_buttons(
    upper_button_position=2.17, #Just some value, not accurate
    lower_button_position=3.5, #Just some value, not accurate
    angular_position=0, #Just some value, not accurate #0 stopni
)

main = trb.add_parachute(
    name="main",
    cd_s=12.72, # cd * parachute area
    trigger=1000,      
    sampling_rate=105,
    lag=6,
    noise=(0, 8.3, 0.5),
    radius=2.25, 
    height=2.25,
    porosity=0.0432,
)

drogue = trb.add_parachute(
    name="drogue",
    cd_s=1.218,
    trigger="apogee", 
    sampling_rate=105,
    lag=1,
    noise=(0, 8.3, 0.5),
    radius=0.76,
    height=0.76,
    porosity=0.0432,
)

# trb.draw()
# trb.plots.drag_curves()

In [ ]:
test_flight = Flight(
    rocket=trb, environment=env, rail_length=15.24, inclination=87, heading=90
    )

In [ ]:
test_flight.prints.out_of_rail_conditions()
test_flight.plots.trajectory_3d()

In [ ]:
print(ex.rail_departure_velocity_in_ft_per_sec(test_flight))

In [ ]:
print(ex.average_thrust_during_rail_phase(test_flight, z4000, trb))

In [ ]:
print(ex.max_static_margin(test_flight))
print(ex.min_static_margin(test_flight))

In [ ]:
# max acceleration in m/s^2 and in g of the whole flight so it gets the main parachute opening as the max
print(test_flight.max_acceleration)
print(ex.max_acceleration(test_flight))
print(ex.max_acceleration_in_g(test_flight))

In [ ]:
# here max during engine work
print(ex.max_acceleration_power_on_in_g(test_flight))

In [ ]:
print(ex.max_velocity_in_ft_per_sec(test_flight))

In [ ]:
print(ex.max_mach_number(test_flight))

In [ ]:
print(ex.max_q_in_psf_and_altitude_in_ft(test_flight))

In [ ]:
print(test_flight.apogee)
print(test_flight.apogee_time)
print(test_flight.apogee*3.28084)
print(ex.max_altitude_in_ft_and_time(test_flight))

In [ ]:
print(ex.max_pitch_moment(test_flight))
print(ex.max_yaw_moment(test_flight))

In [ ]:
print(ex.distance_from_pad(test_flight))

In [ ]:
# calculating damping ratio using logarythmic decrement method, using the peaks of the oscillations after rail departure, here the simple method
t_data, alpha_data, peak_times, peak_values =ex.get_aoa_peaks(test_flight)
damping_ratios, damping_times =ex.calculate_damping_ratios(peak_times=peak_times, peak_values=peak_values)
ex.analyze_and_plot_damping(test_flight)

In [ ]:
#Using more advanced method
time_values, signal_values = ex.get_flight_signal(test_flight, "partial_angle_of_attack")
signal_detrended, amplitude_envelope, instantaneous_omega = ex.process_analytic_signal(time_values, signal_values)
damping_ratios, damping_times = ex.calculate_sliding_damping(time_values, amplitude_envelope, instantaneous_omega)
ex.analyze_advanced_damping(test_flight, signal_name="partial_angle_of_attack")

In [ ]:
# cp vs aoa and cp and cg vs time plots
time, cp_position, cg_position, angle_of_attack = ex.calculate_aero_centers(trb, test_flight)
ex.plot_aerodynamic_stability(trb, test_flight)

In [ ]:
ex.plot_cp_vs_mach_number(trb, test_flight)